In [3]:
# %pip install azure-ai-ml
# %pip install fsspec

In [8]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Data, Datastore
from azure.ai.ml.entities import OneLakeDatastore, OneLakeArtifact
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azureml.fsspec import AzureMachineLearningFileSystem
import pandas as pd
import fsspec

In [5]:
client = MLClient.from_config(credential=DefaultAzureCredential(), path=".")
ws = client.workspaces.get(name=client.workspace_name)
print(ws.name)

Found the config file in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/e4s-4c32g/code/.azureml/config.json


ml_test


In [7]:
store = OneLakeDatastore(
    name="onelake_retry3",
    description="Datastore pointing to an Microsoft fabric artifact.",
    one_lake_workspace_name="6558a001-c860-4020-8ab1-1e5ae1f48456",
    endpoint="onelake.dfs.fabric.microsoft.com",
    artifact = OneLakeArtifact(
        name="1202190f-974b-4175-b882-5ca0c26760f4/Files",
        type="lake_house"
    )
)

client.create_or_update(store)

OneLakeDatastore({'type': <DatastoreType.ONE_LAKE: 'OneLake'>, 'name': 'onelake_retry3', 'description': 'Datastore pointing to an Microsoft fabric artifact.', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/05f2e45e-7783-4a5b-ad4d-c498610f67fb/resourceGroups/rgfabricpoc/providers/Microsoft.MachineLearningServices/workspaces/ml_test/datastores/onelake_retry3', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/e4s-4c32g/code/Users/user07', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7fd227c97730>, 'credentials': <azure.ai.ml.entities._credentials.NoneCredentialConfiguration object at 0x7fd227c973d0>, 'artifact': {'name': '1202190f-974b-4175-b882-5ca0c26760f4/Files', 'type': 'lake_house'}, 'one_lake_workspace_name': '6558a001-c860-4020-8ab1-1e5ae1f48456', 'endpoint': 'onelake.dfs.fabric.microsoft.com'})

### Create Data Asset

In [ ]:
import mltable
from mltable import MLTableHeaders, MLTableFileEncoding, DataType
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential

# create paths to the data files
paths = [{"file": "wasbs://data@azuremlexampledata.blob.core.windows.net/titanic.csv"}]

# create an MLTable from the data files
tbl = mltable.from_delimited_files(
    paths=paths,
    delimiter=",",
    header=MLTableHeaders.all_files_same_headers,
    infer_column_types=True,
    include_path_column=False,
    encoding=MLTableFileEncoding.utf8,
)

# filter out rows undefined ages
tbl = tbl.filter("col('Age') > 0")

# drop PassengerId
tbl = tbl.drop_columns(["PassengerId"])

# ensure survived column is treated as boolean
data_types = {
    "Survived": DataType.to_bool(
        true_values=["True", "true", "1"], false_values=["False", "false", "0"]
    )
}
tbl = tbl.convert_column_types(data_types)

# show the first few records
print(tbl.show())

# save the data loading steps in an MLTable file
mltable_folder = "./titanic"
tbl.save(mltable_folder)

# Connect to the AzureML workspace
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

# Define the Data asset object
my_data = Data(
    path=mltable_folder,
    type=AssetTypes.MLTABLE,
    description="<ADD A DESCRIPTION HERE>",
    name="<NAME OF DATA ASSET>",
    version="<SET VERSION HERE>",
)

# Create the data asset in the workspace
ml_client.data.create_or_update(my_data)